# Feature Processing and Selection

This notebook focuses on exploration using two essential files: the annotations data extracted from the actual screening profile (available in the [IDR repository](https://github.com/IDR/idr0133-dahlin-cellpainting/tree/main/screenA)) and the metadata retrieved from the supplementary section of the [research paper](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-36829-x/MediaObjects/41467_2023_36829_MOESM5_ESM.xlsx).

We explore the number of unique compounds associated with each cell injury and subsequently cross-reference this information with the screening profile. The aim is to assess the feasibility of using the data for training a machine learning model to predict cell injury.

We apply feature selection through [pycytominer](https://github.com/cytomining/pycytominer) to capture the most informative features representing various cellular injury types within the morphology space. Then, we utilize the selected feature profiles for machine learning applications.


In [1]:
import sys
import json
import pathlib
from collections import defaultdict

import pandas as pd
from pycytominer import feature_select

sys.path.append("../../")
from src import utils

In [2]:
# data directory
data_dir = pathlib.Path("../../data").resolve(strict=True)
results_dir = pathlib.Path("../../results").resolve(strict=True)
fs_dir = (results_dir / "0.feature_selection").resolve()
fs_dir.mkdir(exist_ok=True)

# data paths
suppl_meta_path = (data_dir / "41467_2023_36829_MOESM5_ESM.csv.gz").resolve(strict=True)
screen_anno_path = (data_dir / "idr0133-screenA-annotation.csv.gz").resolve(strict=True)

# load data
image_profile_df = pd.read_csv(screen_anno_path)
meta_df = image_profile_df[image_profile_df.columns[:31]]
compounds_df = meta_df[["Compound Name", "Compound Class"]]

suppl_meta_df = pd.read_csv(suppl_meta_path)
cell_injury_df = suppl_meta_df[["Cellular injury category", "Compound alias"]]

In [3]:
# get the control
control_df = image_profile_df.loc[image_profile_df["Compound Name"] == "DMSO"]
control_df.insert(0, "injury_type", "Control")

# display
print("Shape of the control:", control_df.shape)
control_df.head()

Shape of the control: (9855, 404)


,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,Experimental Condition [Treatment time (h)],...,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumVariance_DNA_20_0
0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.000098,0.057244,0.160847,-0.083034,-0.023290,-0.066369,-0.015235,-0.035909,-0.013321,-0.032067
1,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.025857,0.099848,0.017477,0.021300,0.058137,-0.097280,-0.073545,-0.044883,-0.089842,-0.015240
2,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.041060,0.119247,0.111741,0.041592,0.224199,-0.088845,0.000327,-0.003115,0.016075,-0.014406
3,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.022156,0.036473,-0.013141,0.008690,0.060860,0.044924,0.040528,0.070877,0.038779,0.072871
4,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.007213,0.023068,0.110361,0.054405,0.030157,0.066480,0.038910,0.048559,0.050371,0.056829


In [4]:
# getting profiles based on injury and compound type
injury_and_compounds = defaultdict(list)
for injury, compound in cell_injury_df.values.tolist():
    injury_and_compounds[injury].append(compound)

# cross reference selected injury and associated components into the screen profile
injury_profiles = []
for injury_type, compound_list in injury_and_compounds.items():
    sel_profile = image_profile_df[
        image_profile_df["Compound Name"].isin(compound_list)
    ]
    sel_profile.insert(0, "injury_type", injury_type)
    injury_profiles.append(sel_profile)

In [5]:
# creating a dataframe that contains stratified screen Data
injured_df = pd.concat(injury_profiles)

# drop wells that do not have an injury
injured_df = injured_df.dropna(subset="injury_type").reset_index(drop=True)
print("Number of plates", len(injured_df["Plate"].unique()))

# display df
print("shape:", injured_df.shape)
injured_df.head()

Number of plates 84
shape: (6848, 404)


,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,Experimental Condition [Treatment time (h)],...,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumVariance_DNA_20_0
0,Cytoskeletal,BR00110363,E17,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.561075,0.139535,0.188096,-1.035562,0.655389,0.182888,-0.004066,0.130472,-0.418286,0.283484
1,Cytoskeletal,BR00110363,E18,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.642707,0.052501,0.130166,-1.304556,0.438742,0.187985,0.088121,0.289709,-0.451626,0.461128
2,Cytoskeletal,BR00110363,E19,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.599857,0.184587,0.111444,-1.462714,0.821791,0.229490,0.121207,0.165713,-0.342221,0.388047
3,Cytoskeletal,BR00110363,E20,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.513671,0.137843,0.165498,-1.005157,0.264772,0.169579,0.142331,0.264883,-0.161366,0.337277
4,Cytoskeletal,BR00110363,E21,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.402869,0.083364,0.181626,-1.068167,0.469826,0.411077,0.427186,0.458690,-0.012347,0.658387


In [6]:
# seperating meta and feature columns
meta = injured_df.columns.tolist()[:32]
features = injured_df.columns.tolist()[32:]

In [7]:
# dropping samples that have at least 1 NaN
injured_df = utils.drop_na_samples(profile=injured_df, features=features, cut_off=0)

# display
print("Shape after removing samples: ", injured_df.shape)
injured_df.head()

Shape after removing samples:  (6846, 404)


,Compound Manuscript Number,Characteristics [Organism],Mahalanobis distance significant,Term Source 2 Accession,Term Source 2 REF,Compound Purity (%),Compound BRD,Compound PubChem URL,Relative well cellcount significant,Compound Name,...,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumVariance_DNA_20_0
0,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.561075,0.139535,0.188096,-1.035562,0.655389,0.182888,-0.004066,0.130472,-0.418286,0.283484
1,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.642707,0.052501,0.130166,-1.304556,0.438742,0.187985,0.088121,0.289709,-0.451626,0.461128
2,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.599857,0.184587,0.111444,-1.462714,0.821791,0.229490,0.121207,0.165713,-0.342221,0.388047
3,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.513671,0.137843,0.165498,-1.005157,0.264772,0.169579,0.142331,0.264883,-0.161366,0.337277
4,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.402869,0.083364,0.181626,-1.068167,0.469826,0.411077,0.427186,0.458690,-0.012347,0.658387


In [8]:
# setting feature selection operations
all_operations = [
    "variance_threshold",
    "correlation_threshold",
    "drop_na_columns",
    "blocklist",
    "drop_outliers",
]

# Applying feature selection using pycytominer
fs_injury_df = feature_select(
    profiles=injured_df,
    features=features,
    operation=all_operations,
    freq_cut=0.05,
    corr_method="pearson",
    corr_threshold=0.90,
    na_cutoff=0.0,
    outlier_cutoff=100,
)

In [9]:
print("Feature selected profile shape:", fs_injury_df.shape)
fs_injury_df.head()

Feature selected profile shape: (6846, 378)


,Compound Manuscript Number,Characteristics [Organism],Mahalanobis distance significant,Term Source 2 Accession,Term Source 2 REF,Compound Purity (%),Compound BRD,Compound PubChem URL,Relative well cellcount significant,Compound Name,...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
0,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.097460,0.561075,0.139535,0.188096,-1.035562,0.655389,0.182888,-0.004066,0.130472,0.283484
1,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.065539,0.642707,0.052501,0.130166,-1.304556,0.438742,0.187985,0.088121,0.289709,0.461128
2,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.101799,0.599857,0.184587,0.111444,-1.462714,0.821791,0.229490,0.121207,0.165713,0.388047
3,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,0.072294,0.513671,0.137843,0.165498,-1.005157,0.264772,0.169579,0.142331,0.264883,0.337277
4,2,Homo sapiens,No,EFO_0002869,EFO,99.4,BRD-K12539581-001-24-5,https://pubchem.ncbi.nlm.nih.gov/compound/4122,No,Nocodazole,...,-0.139160,0.402869,0.083364,0.181626,-1.068167,0.469826,0.411077,0.427186,0.458690,0.658387


In [10]:
# update the control with the retained features in the injury_fs_profile
control_df = control_df[fs_injury_df.columns]

# display
print(
    "Shape of control after using feature retained from injury_fs profile",
    control_df.shape,
)
control_df.head()

Shape of control after using feature retained from injury_fs profile (9855, 378)


,Compound Manuscript Number,Characteristics [Organism],Mahalanobis distance significant,Term Source 2 Accession,Term Source 2 REF,Compound Purity (%),Compound BRD,Compound PubChem URL,Relative well cellcount significant,Compound Name,...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
0,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,-0.011258,0.000098,0.057244,0.160847,-0.083034,-0.023290,-0.066369,-0.015235,-0.035909,-0.032067
1,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.064689,0.025857,0.099848,0.017477,0.021300,0.058137,-0.097280,-0.073545,-0.044883,-0.015240
2,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.020937,0.041060,0.119247,0.111741,0.041592,0.224199,-0.088845,0.000327,-0.003115,-0.014406
3,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.006589,0.022156,0.036473,-0.013141,0.008690,0.060860,0.044924,0.040528,0.070877,0.072871
4,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,-0.028361,0.007213,0.023068,0.110361,0.054405,0.030157,0.066480,0.038910,0.048559,0.056829


In [11]:
# concat both the injury and control together and make this is that feature selected profile
fs_profile = pd.concat([control_df, fs_injury_df])

# save and display
fs_profile.to_csv(
    fs_dir / "cell_injury_profile_fs.csv.gz",
    index=False,
    compression="gzip",
)
print("Shape of fs_profile", fs_profile.shape)
fs_profile.head()

Shape of fs_profile (16701, 378)


,Compound Manuscript Number,Characteristics [Organism],Mahalanobis distance significant,Term Source 2 Accession,Term Source 2 REF,Compound Purity (%),Compound BRD,Compound PubChem URL,Relative well cellcount significant,Compound Name,...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
0,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,-0.011258,0.000098,0.057244,0.160847,-0.083034,-0.023290,-0.066369,-0.015235,-0.035909,-0.032067
1,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.064689,0.025857,0.099848,0.017477,0.021300,0.058137,-0.097280,-0.073545,-0.044883,-0.015240
2,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.020937,0.041060,0.119247,0.111741,0.041592,0.224199,-0.088845,0.000327,-0.003115,-0.014406
3,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.006589,0.022156,0.036473,-0.013141,0.008690,0.060860,0.044924,0.040528,0.070877,0.072871
4,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,-0.028361,0.007213,0.023068,0.110361,0.054405,0.030157,0.066480,0.038910,0.048559,0.056829


In [12]:
# saving feature space
cell_injury_meta, cell_injury_feats = utils.split_meta_and_features(
    fs_profile, metadata_tag=False
)

# saving info of feature space
jump_feature_space = {
    "name": "cell_injury",
    "n_plates": len(fs_profile["Plate"].unique()),
    "n_meta_features": len(cell_injury_meta),
    "n_features": len(cell_injury_feats),
    "meta_features": cell_injury_meta,
    "features": cell_injury_feats,
}

# save json file
with open(fs_dir / "cell_injury_feature_space.json", mode="w") as f:
    json.dump(jump_feature_space, f)

# display
print("NUmber of plates", len(fs_profile["Plate"].unique()))
print("Number of meta features", len(cell_injury_meta))
print("Number of features", len(cell_injury_feats))

NUmber of plates 84
Number of meta features 32
Number of features 346


In [13]:
# setting which injr
cell_injuries = fs_profile["injury_type"].unique()
print("number of cell injury types", len(cell_injuries))
cell_injuries

number of cell injury types 15


array(['Control', 'Cytoskeletal', 'Hsp90', 'Kinase', 'Genotoxin',
       'Miscellaneous', 'Redox', 'HDAC', 'mTOR', 'Proteasome', 'Saponin',
       'Mitochondria', 'Ferroptosis', 'Tannin', 'Nonspecific reactive'],
      dtype=object)

Let see how many overlapping feature between the cell injury and the jump. In addition, the order of the overlapping featues should be maintained


In [14]:
jump_feature_space = utils.load_json_file(
    data_dir / "JUMP_data/jump_feature_space.json"
)

# get feature space of both datasets
jump_feats = set(jump_feature_space["features"])
cell_injury_feats = set(cell_injury_feats)

# find how many are overlapping
overlapping = jump_feats.intersection(cell_injury_feats)

print("Number of features in Cell Injury", len(cell_injury_feats))
print("Number of features in JUMP", len(jump_feats))
print("Number of overlapping feats", len(overlapping))
print(
    "Number of features that are not overlapping",
    len(cell_injury_feats) - len(overlapping),
)

Number of features in Cell Injury 346
Number of features in JUMP 5792
Number of overlapping feats 207
Number of features that are not overlapping 139


In [15]:
# select the columns that are in the feature space profile
selected_cols = []
for col_name in fs_profile.columns:
    if col_name in overlapping:
        selected_cols.append(col_name)

# create a new dataframe that contains the feature selected profile with only overlapping features
# augment overlapping_fs_profile with the metadata
overlapping_fs_profile = fs_profile[selected_cols]
overlapping_fs_profile = pd.concat(
    [fs_profile[cell_injury_meta], overlapping_fs_profile], axis=1
)

# saving info of feature space
overlapping_fs_profile_feature_space = {
    "name": "cell_injury_overlapping",
    "n_plates": len(fs_profile["Plate"].unique()),
    "n_meta_features": len(cell_injury_meta),
    "n_features": len(selected_cols),
    "meta_features": cell_injury_meta,
    "features": selected_cols,
}

# save json file
with open(fs_dir / "cell_injury_overlapping_feature_space.json", mode="w") as f:
    json.dump(jump_feature_space, f)

# save overlapping profile
overlapping_fs_profile.to_csv(
    fs_dir / "overlapping_cell_injury_profile_fs.csv.gz",
    index=False,
    compression="gzip",
)

print("Shape of overlapping fs profile", overlapping_fs_profile.shape)
overlapping_fs_profile.head()

Shape of overlapping fs profile (16701, 239)


,Compound Manuscript Number,Characteristics [Organism],Mahalanobis distance significant,Term Source 2 Accession,Term Source 2 REF,Compound Purity (%),Compound BRD,Compound PubChem URL,Relative well cellcount significant,Compound Name,...,Nuclei_RadialDistribution_RadialCV_ER_2of4,Nuclei_RadialDistribution_RadialCV_ER_3of4,Nuclei_RadialDistribution_RadialCV_ER_4of4,Nuclei_RadialDistribution_RadialCV_Mito_1of4,Nuclei_RadialDistribution_RadialCV_Mito_2of4,Nuclei_RadialDistribution_RadialCV_Mito_3of4,Nuclei_RadialDistribution_RadialCV_Mito_4of4,Nuclei_RadialDistribution_RadialCV_RNA_1of4,Nuclei_RadialDistribution_RadialCV_RNA_2of4,Nuclei_RadialDistribution_RadialCV_RNA_3of4
0,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.061971,0.024875,-0.014376,0.082153,0.049476,0.013555,0.041612,0.042220,0.126820,0.077685
1,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.041521,-0.000281,-0.007509,0.016360,0.033403,-0.015250,0.002146,0.076559,0.076925,0.051081
2,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.039369,0.000791,-0.031197,-0.065423,-0.087133,-0.086666,-0.155381,0.083111,0.093994,0.040758
3,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.020103,0.015091,-0.013667,0.010676,-0.006710,0.003637,-0.018047,-0.015474,-0.017643,0.011583
4,502,Homo sapiens,No,EFO_0002869,EFO,NaN,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/679,No,DMSO,...,0.000525,-0.026855,-0.023019,-0.058229,-0.064460,-0.046734,-0.071742,-0.033140,-0.040650,-0.019959
